## Table of Contents
* [Introduction](#introduction)
* [Ensuring that you have the necessary packages to go through this tutorial <a name="system_check"></a>](#ensuring-that-you-have-the-necessary-packages-to-go-through-this-tutorial-a-name"systemcheck"a)
* [Initializing a system - FCC lattice](#initializing-a-system---fcc-lattice)

## Introduction
The purpose of this notebook is to provide a starting place for running HOOMD-blue molecular dynamics simulations. It is recommended that you read the official HOOMD-blue tutorial (<https://hoomd-blue.readthedocs.io/en/v3.0.1/tutorial/01-Introducing-Molecular-Dynamics/01-Molecular-Dynamics-Simulations.html>) that contains some basic information that can help you to understand the basic set-up of the simulation scripts before going through this notebook. This document will hopefully help you to remember and get more familiar with the HOOMD-blue classes/functions and Python syntax to use them.

## Ensuring that you have the necessary packages to go through this tutorial <a name="system_check"></a>
Going through this notebook will require you to have certain Python packages installed on your machine. Assuming you have them set-up within your conda environment, activate the environment and check for the following packages with `conda list | grep "*package1 name*\|*package2 name*\|*package3 name*...`.
<ul>
    <li>hoomd</li>
    <li>freud</li>
    <li>gsd</li>
    <li>matplotlib</li>
</ul>

The output should resemble the following:
>freud                     2.8.0           py310he7ab2d0_0    conda-forge

>gsd                       2.5.2           py310h7f5fb2b_0    conda-forge

>hoomd                     3.0.0           cpu_py310h89b14bd_1    conda-forge

>matplotlib                3.5.1           py310hecd8cb5_1

>matplotlib-base           3.5.1           py310hfb0c5b7_1

>matplotlib-inline         0.1.2              pyhd3eb1b0_2

If all four packages are installed, start up an interactive Python session with `python` and type the following for the basic functions of the simulation to ensure that your packages are functioning correctly without conflicts.

```python
import hoomd, freud, gsd.hoomd, matplotlib.pyplot as plt
hoomd.device.CPU()
freud.Box.cube(10)
gsd.hoomd.Snapshot()
plt.figure()
```

If all works without any error, proceed to the next section. If not, you may need to reinstall the malfunctioning packages.

## Initializing a system - FCC lattice